<a href="https://colab.research.google.com/github/vegikumar/text-generation/blob/main/text%20generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
file = open("frankeinstein.txt").read()

In [3]:
def tokenize_words(input):
  input = input.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return " ".join(filtered)

processed_inputs = tokenize_words(file)

In [4]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [5]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total no of characters:", input_len)
print ("Total vocab",vocab_len)

Total no of characters: 1365
Total vocab 26


In [6]:
seq_length = 100
x_data = []
y_data = []

In [7]:
for i in range(0, input_len - seq_length, 1):
  in_seq = processed_inputs[i:i + seq_length]
  out_seq = processed_inputs[i + seq_length]
  x_data.append([char_to_num[char]for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total patterns:", n_patterns)

Total patterns: 1265


In [8]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [9]:
y = np_utils.to_categorical(y_data)

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [13]:
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
5/5 [==============================] - ETA: 0s - loss: 3.1352
Epoch 00001: loss improved from inf to 3.13516, saving model to model_weights_saved.hdf5
5/5 [==============================] - 22s 3s/step - loss: 3.1352
Epoch 2/4
5/5 [==============================] - ETA: 0s - loss: 2.9471
Epoch 00002: loss improved from 3.13516 to 2.94713, saving model to model_weights_saved.hdf5
5/5 [==============================] - 16s 3s/step - loss: 2.9471
Epoch 3/4
5/5 [==============================] - ETA: 0s - loss: 2.9361
Epoch 00003: loss improved from 2.94713 to 2.93612, saving model to model_weights_saved.hdf5
5/5 [==============================] - 16s 3s/step - loss: 2.9361
Epoch 4/4
5/5 [==============================] - ETA: 0s - loss: 2.9215
Epoch 00004: loss improved from 2.93612 to 2.92154, saving model to model_weights_saved.hdf5
5/5 [==============================] - 16s 3s/step - loss: 2.9215


In [14]:
filename = 'model_weights_saved.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [16]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("RAndom seed: ")
print("\".".join([num_to_char[value] for value in pattern]),"\"")

RAndom seed: 
e".s".i".d".e". ".g".r".e".a".t". ".u".n".e".x".p".l".o".r".e".d". ".o".c".e".a".n". ".t".r".u".t".h". ".s".u".c".c".e".s".s".o".r".s". ".b".r".a".n".c".h". ".n".a".t".u".r".a".l". ".p".h".i".l".o".s".o".p".h".y". ".a".c".q".u".a".i".n".t".e".d". ".a".p".p".e".a".r".e".d". ".e".v".e".n". ".b".o".y "


In [17]:
for i in range(1000):
  x = numpy.reshape(pattern, (1,len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee